In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

In [ ]:
class ImageCaptioner:
    def __init__(self):
        self.model_caption = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        self.text = "a photography of"
    def run(self, img_path):

      # conditional image captioning

      inputs = self.processor(img_path, self.text, return_tensors="pt")

      out = self.model_caption.generate(**inputs)
      return self.processor.decode(out[0], skip_special_tokens=True)

In [ ]:
img_captioner = ImageCaptioner()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
output_7 = img_captioner.run('/content/images/image_caption7.jpg')
print(output_7)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a man is painting a wall with a group of children


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

In [ ]:
class Translator:
    def __init__(self):
      self.model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

      self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
      self.model_Llama = AutoModelForCausalLM.from_pretrained(
      self.model_id,
      torch_dtype=torch.bfloat16,
      device_map="auto",
      )

    def run(self, text_english):
      messages = [
          {"role": "system", "content": "You are a skilled translator tasked with translating English text into Arabic. Your output must be strictly in Arabic, without any English words, symbols, or additional explanations. Translate only the provided text and do not include any notes."},
          {"role": "user", "content": text_english},
      ]

      input_ids = self.tokenizer.apply_chat_template(
          messages,
          add_generation_prompt=True,
          return_tensors="pt"
      ).to(self.model_Llama.device)

      terminators = [
          self.tokenizer.eos_token_id,
          self.tokenizer.convert_tokens_to_ids("<|eot_id|>")
      ]

      outputs = self.model_Llama.generate(
          input_ids,
          max_new_tokens=256,
          eos_token_id=terminators,
          do_sample=True,
          temperature=0.6,
          top_p=0.9,
      )
      response = outputs[0][input_ids.shape[-1]:]
      return self.tokenizer.decode(response, skip_special_tokens=True)


In [ ]:
translator = Translator()

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
translated_sentence = translator.run()
print(translated_sentence)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


صورة رجل يرسم جدار مع مجموعة من الأطفال


In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch
import soundfile as sf
from IPython.display import Audio

In [ ]:
class TTS:
  def __init__(self):
    self.model = VitsModel.from_pretrained("wasmdashai/vits-ar")
    self.tokenizer = AutoTokenizer.from_pretrained("wasmdashai/vits-ar")

  def run(self, text):
    inputs = self.tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        full_generation = self.model(**inputs)

    full_generation_waveform = full_generation.waveform.cpu().numpy().reshape(-1)

    # Saving the audio waveform as a WAV file
    sf.write("speech.wav", full_generation_waveform, samplerate=self.model.config.sampling_rate, format='wav')
    Audio(full_generation_waveform, rate=self.model.config.sampling_rate)

In [ ]:
tts = TTS()

In [ ]:
tts.run(translated_sentence)

In [ ]:
from transformers import AutoProcessor, SeamlessM4Tv2Model
import torchaudio
from IPython.display import Audio
import scipy


/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [ ]:
class TTS_2:
  def __init__(self):
    self.processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
    self.model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

  def run(self, text):
    # Change target language to "arb" (Modern Standard Arabic)
    text_inputs = self.processor(text = text, src_lang="arb", return_tensors="pt")
    audio_array_from_text = self.model.generate(**text_inputs, tgt_lang="arb")[0].cpu().numpy().squeeze()

    audio, orig_freq =  torchaudio.load("https://www2.cs.uic.edu/~i101/SoundFiles/preamble10.wav")
    audio =  torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=16_000) # must be a 16 kHz waveform array
    audio_inputs = self.processor(audios=audio, return_tensors="pt")
    # Change target language to "arb" (Modern Standard Arabic)
    audio_array_from_audio = self.model.generate(**audio_inputs, tgt_lang="arz")[0].cpu().numpy().squeeze()

    # Access sampling_rate from model.config
    sample_rate = self.model.config.sampling_rate
    scipy.io.wavfile.write("out_from_text.wav", rate=sample_rate, data=audio_array_from_text)

    return Audio(audio_array_from_text, rate=sample_rate)

    # scipy.io.wavfile.write("out_from_audio.wav", rate=sample_rate, data=audio_array_from_audio)

In [ ]:
tts_2 = TTS_2()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
audio = tts_2.run(translated_sentence)
audio

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


ValueError: `tgt_lang=arz` is not supported by this model.
                    Please specify a `tgt_lang` in arb,ben,cat,ces,cmn,cym,dan,deu,eng,est,fin,fra,hin,ind,ita,jpn,kan,kor,mlt,nld,pes,pol,por,ron,rus,slk,spa,swe,swh,tam,tel,tgl,tha,tur,ukr,urd,uzn,vie. Note that SeamlessM4Tv2 supports
                    more languages for text translation than for speech synthesis.

In [ ]:
class ImageToVoice:
  def __init__(self):
    self.img_captioner = ImageCaptioner()
    self.translator = Translator()
    # self.tts = TTS()
    self.tts_2 = TTS_2()

  def run(self, img_path):
    output = self.img_captioner.run(img_path)
    translated_sentence = self.translator.run(output)
    return self.tts_2.run(translated_sentence)

In [ ]:
img_to_voice = ImageToVoice()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [ ]:
# Create the Gradio interface

image = gr.Image(type="pil", label="Upload Image") # removed gr.inputs
voice = gr.Audio(type="filepath", label="Generated Voice") # removed gr.outputs

interface = gr.Interface(
    fn=img_to_voice.run,
    inputs=image,
    outputs=voice,
    title="Image to Speech Generator",
    description="Upload an image and get an audio file generated from it."
)

interface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://262a9463dda89ff24d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.ge

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://262a9463dda89ff24d.gradio.live


In [ ]:
from transformers import AutoProcessor, SeamlessM4Tv2Model
import torchaudio

processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

# from text
text_inputs = processor(text = translated_sentence, src_lang="arz", return_tensors="pt")
audio_array_from_text = model.generate(**text_inputs, tgt_lang="arz")[0].cpu().numpy().squeeze()

# from audio
audio, orig_freq =  torchaudio.load("https://www2.cs.uic.edu/~i101/SoundFiles/preamble10.wav")
audio =  torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=16_000) # must be a 16 kHz waveform array
audio_inputs = processor(audios=audio, return_tensors="pt")
audio_array_from_audio = model.generate(**audio_inputs, tgt_lang="arz")[0].cpu().numpy().squeeze()


In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch
import scipy

model = VitsModel.from_pretrained("SeyedAli/Arabic-Speech-synthesis")
tokenizer = AutoTokenizer.from_pretrained("SeyedAli/Arabic-Speech-synthesis")

text = "some example text in the Arabic language"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform

scipy.io.wavfile.write("techno.wav", rate=model.config.sampling_rate, data=output)